In [2]:
import weaviate
import os
import requests
import json

from markdownify import markdownify as md
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
client = weaviate.connect_to_local()

print(client.is_ready())  # Should print: `True`

client.close()

True


In [5]:
from weaviate.classes.config import Configure

client = weaviate.connect_to_local()


questions = client.collections.create(
    name="test",
    vectorizer_config=Configure.Vectorizer.text2vec_openai(),   # Configure the OpenAI embedding integration
    generative_config=Configure.Generative.openai(),  
)

    
client.close()

UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name Test already exists'}]}.

In [19]:
# Get endpoints for a certain tag
#tag = "Metric"

obp_base_url = os.getenv("OBP_BASE_URL")
print(obp_base_url)
obp_version = "v5.1.0"

#swagger_url = "{}/obp/v5.1.0/resource-docs/{}/swagger?tags={}".format(os.getenv("OBP_BASE_URL"), obp_version, tag)
swagger_url = "{}/obp/v5.1.0/resource-docs/{}/swagger?content=static".format(obp_base_url, obp_version)

# check validator is working

import logging
from swagger_spec_validator import validate_spec_url

#logging.basicConfig(level=logging.DEBUG)
print("Validating Swagger Spec")
try:
    validate_spec_url(swagger_url)
except Exception as e:
    print(f"Could not validate swagger spec: {e}")
    
swagger_response = requests.get(swagger_url)

if swagger_response.status_code >= 300:
    raise Exception(f"Swagger endpoint malfunction: {swagger_response.json()}")
swagger_json = swagger_response.json()

# Fix three very broken refs
#swagger_json['definitions']['ResourceDocJson']['properties']['success_response_body']['$ref'] = '#/definitions/EmptyClassJson'
#swagger_json['definitions']['ResourceDocJson']['properties']['example_request_body']['$ref'] = '#/definitions/EmptyClassJson'
#swagger_json['definitions']['ConsentJsonV500']['properties']['account_access']['$ref'] = '#/definitions/EmptyClassJson'
#swagger_json['definitions']['ConsentJWT']['properties']['entitlements']['items']['$ref'] = '#/definitions/EntitlementsJsonV400'

# Fix duplicate operationID
#swagger_json['paths']['/obp/v5.1.0/dynamic-registration/consumers']['post']['operationId'] = 'dynamicRegistrationCreateConsumer'

# Fix duplicate tags
#swagger_json['paths']['/obp/v5.1.0/banks']['get']['tags'] = list(set(swagger_json['paths']['/obp/v5.1.0/banks']['get']['tags']))

print(len(swagger_json['paths']))

#### IMPORTANT: UNTIL THE SWAGGER SPEC CHECKING ON OBP SIDE IS FIXED, DO NOT UNCOMMENT THIS
# We will do development with a static i.e. not dynamically updated spec until this is the case

#with open('swagger_spec.json', 'w') as f:
#    json.dump(swagger_json, f, indent =3)

https://apisandbox.openbankproject.com
Validating Swagger Spec


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): apisandbox.openbankproject.com:443


Could not validate swagger spec: ('The read operation timed out', TimeoutError('The read operation timed out'))


DEBUG:urllib3.connectionpool:https://apisandbox.openbankproject.com:443 "GET /obp/v5.1.0/resource-docs/v5.1.0/swagger?content=static HTTP/1.1" 400 189


Exception: Swagger endpoint malfunction: {'code': 400, 'message': 'OBP-50000: Unknown Error. Can not convert internal swagger file. <- JArray type should not be empty. <- java.lang.RuntimeException: JArray type should not be empty.'}

In [ ]:
swagger_json

In [43]:
from prance import ResolvingParser

def recursion_handler(recursion_limit, parsed_url, refs):
    print(recursion_limit, parsed_url, refs)
    return {'$ref': '#'+parsed_url.fragment}

parser = ResolvingParser('swagger_spec.json', backend ="swagger-spec-validator", recursion_limit_handler=recursion_handler)

In [44]:
resolved_spec_dict = parser.specification

In [45]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Define an endpoint to summarize the descriptions
prompt = ChatPromptTemplate.from_template(
    "Summarise the following OpenAPI endpoint description (written in html) in plain text with less that 1024 characters:\n\n{raw_description}"
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

endpoint_summary_chain = prompt | llm

/home/nemo/anaconda3/envs/opey/lib/python3.13/site-packages/pydantic/v1/typing.py:68: DeprecationWarning: Failing to pass a value to the 'type_params' parameter of 'typing.ForwardRef._evaluate' is deprecated, as it leads to incorrect behaviour when calling typing.ForwardRef._evaluate on a stringified annotation that references a PEP 695 type parameter. It will be disallowed in Python 3.15.
  return cast(Any, type_)._evaluate(globalns, localns, recursive_guard=set())
/home/nemo/anaconda3/envs/opey/lib/python3.13/site-packages/pydantic/v1/typing.py:68: DeprecationWarning: Failing to pass a value to the 'type_params' parameter of 'typing.ForwardRef._evaluate' is deprecated, as it leads to incorrect behaviour when calling typing.ForwardRef._evaluate on a stringified annotation that references a PEP 695 type parameter. It will be disallowed in Python 3.15.
  return cast(Any, type_)._evaluate(globalns, localns, recursive_guard=set())
DEBUG:httpx:load_ssl_context verify=True cert=None trust_e

In [46]:
from http.client import responses

def response_code_to_string(code: int) -> str:
    """
    Takes a HTTP response status code i.e. 200 and prepends its descriptor to it i.e.
    200 becomes OK_200
    404 becomes NotAuthorized_404
    """
    descriptor = responses[code]

    # Turn the descriptors into camelCase
    words = descriptor.split()
    if len(words) > 1:
        camel_case_descriptor = words[0].lower() + ''.join(w.title() for w in words[1:])
    else:
        camel_case_descriptor = words[0].lower()
    
    return f"{camel_case_descriptor}_{code}"

test_codes = [100, 200, 301, 202, 404, 500]
for code in test_codes:
    result = response_code_to_string(code)
    print(f"{code} -> {result}")


100 -> continue_100
200 -> ok_200
301 -> movedPermanently_301
202 -> accepted_202
404 -> notFound_404
500 -> internalServerError_500


In [47]:
# Count the total number of endpoints, this is for the progress bar
total_endpoints_count = 0
for path, endpoint in resolved_spec_dict["paths"].items():
    for method, properties in endpoint.items():
        total_endpoints_count +=1
        
print(total_endpoints_count)

538


In [48]:
from weaviate.util import generate_uuid5

client = weaviate.connect_to_local()
test_collection = client.collections.get('test')

documents = []

for path, endpoint in resolved_spec_dict["paths"].items():
    for method, properties in endpoint.items():
        
        
        
        # Skip dynamic resources (this will not be needed when the swagger endpoint is updated to have ?static=false)
        if 'Dynamic-Entity' in properties["tags"]:
            continue
            
        else:
            
            props = properties.copy()
            
            # Summarize the description of the endpoint in markdown
            #summary_chain_response = endpoint_summary_chain.invoke({"raw_description": properties['description']})
            #props['description'] = summary_chain_response.content
            
            #endpoint_object = {"path": path, "method": method, "tags": props["tags"], "schema": str(props)}
            endpoint_object = props

            # Change the description from HTML to markdown format
            props['description'] = md(props['description'])
            # Generate deterministic UUID from OperationID
            endpoint_uuid = generate_uuid5(props['operationId'])

            #Weaviate does not like straight response codes as object keys i.e. 200 or 404 so we need to change that
            responses_with_new_keys = {}
            print(endpoint_object['responses'])
            for response_code, response_schema in endpoint_object['responses'].copy().items():
                
                stringified_response_code = response_code_to_string(int(response_code)) 
                
                responses_with_new_keys[stringified_response_code] = response_schema
                print(responses_with_new_keys)

            # Replace all keys
            endpoint_object['responses'] = responses_with_new_keys
            
            
            documents.append(endpoint_object)

            test_collection.data.insert(
                properties = endpoint_object,
                uuid=endpoint_uuid
            )

            
            
            print(f"{method.upper()} {path} processed...")

client.close()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=False http2=False
DEBUG:httpx:load_verify_locations cafile='/home/nemo/anaconda3/envs/opey/lib/python3.13/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=8080 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x7c8a4fd11ef0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 404, b'Not Found', [(b'Access-Control-Allow-Headers', b'Content-Type, Authorization, Batch, X-Openai-Api-Key, X-Openai-Organization,

{'201': {'description': 'Success', 'schema': {'type': 'object', 'properties': {'is_valid': {'type': 'boolean', 'example': 'true'}, 'details': {'type': 'object', 'properties': {'bank_routings': {'type': 'array', 'items': {'type': 'object', 'properties': {'scheme': {'type': 'string', 'example': 'BIC'}, 'address': {'type': 'string', 'example': 'SOGEDEFF'}}, 'required': ['scheme', 'address']}}, 'bank': {'type': 'string', 'example': 'Societe Generale'}, 'branch': {'type': 'string', 'example': ''}, 'address': {'type': 'string', 'example': 'Neue mainzer strasse 46-50'}, 'city': {'type': 'string', 'example': 'Frankfurt am Main'}, 'postcode': {'type': 'string', 'example': '60311'}, 'phone': {'type': 'string', 'example': ''}, 'country': {'type': 'string', 'example': 'Germany'}, 'attributes': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string', 'example': 'country_iso'}, 'value': {'type': 'string', 'example': ''}}, 'required': ['name', 'value']}}}, 'required': [

UnexpectedStatusCodeError: Object was not added! Unexpected status code: 500, with response body: {'error': [{'message': 'update vector: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY'}]}.

In [38]:
documents[0]

IndexError: list index out of range

In [12]:
client = weaviate.connect_to_local()
test_collection = client.collections.get('Test')
import logging
with test_collection.batch.dynamic() as batch:
    for document in documents:
        obj_uuid = generate_uuid5(document['operationId'])
        batch.add_object(
            properties=document,
            uuid=obj_uuid
        )



DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=False http2=False
DEBUG:httpx:load_verify_locations cafile='/home/nemo/anaconda3/envs/opey/lib/python3.13/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=8080 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x7c8a2abd1630>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 404, b'Not Found', [(b'Access-Control-Allow-Headers', b'Content-Type, Authorization, Batch, X-Openai-Api-Key, X-Openai-Organization,

In [14]:
client = weaviate.connect_to_local()

response = client.collections.list_all(simple=True)

print(response)

test_collection = client.collections.get('Test')
all_docs = test_collection.query.fetch_objects()

client.close()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=False http2=False
DEBUG:httpx:load_verify_locations cafile='/home/nemo/anaconda3/envs/opey/lib/python3.13/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=8080 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x7c8a4c3baed0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 404, b'Not Found', [(b'Access-Control-Allow-Headers', b'Content-Type, Authorization, Batch, X-Openai-Api-Key, X-Openai-Organization,

{'Test': _CollectionConfigSimple(name='Test', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.OPENAI: 'generative-openai'>, model={}), properties=[], references=[], reranker_config=None, vectorizer_config=_VectorizerConfig(vectorizer=<Vectorizers.TEXT2VEC_OPENAI: 'text2vec-openai'>, model={'baseURL': 'https://api.openai.com', 'model': 'text-embedding-3-small'}, vectorize_collection_name=True), vectorizer=<Vectorizers.TEXT2VEC_OPENAI: 'text2vec-openai'>, vector_config=None)}


In [15]:
all_docs.objects

[]